# Status #

- [ ] Outline
- [ ] Introduction
- [ ] Exercise 1
  - [x] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Exercise 2
  - [ ] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Exercise 3
  - [ ] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Exercise 4
  - [ ] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Conclusion

# Introduction #

Get started by running the code cell below.

In [ ]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.computer_vision.ex5 import *

from cv_prelude import *

# Design a Convnet #

Here is a diagram of the model we want to implement:

<figure>
<!-- <img src="./images/2-convmodel-2.png" width="250" alt="Diagram of a convolutional model."> -->
<img src="https://i.imgur.com/Rc59Gwj.png" width="250" alt="Diagram of a convolutional model.">
</figure>

### 1) Define Model

We again have three convolutional blocks, but this time some of them have several convolutional layers. Complete the definition of the model given in the diagram.

In [ ]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

model = Sequential([
    # Block One
    layers.Conv2D(filters=64, kernel_size=5, activation='relu', padding='same',
                  input_shape=[192, 192, 3]),
    layers.MaxPool2D(),

    # Block Two
    # YOUR CODE HERE: Add the missing layers
    ____

    # Block Three
    layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='same'),
    layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='same'),
    layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Head
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])
q_1.check()

In [ ]:
#%%RM_IF(PROD)%%
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

model = Sequential([
    # Block One
    layers.Conv2D(filters=64, kernel_size=5, activation='relu', padding='same',
                  input_shape=[192, 192, 3]),
    layers.MaxPool2D(),

    # Block Two
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Block Three
    layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='same'),
    layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='same'),
    layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Head
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])
q_1.assert_check_passed()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_1.hint()
#_COMMENT_IF(PROD)_
q_1.solution()

### 2) Train

Compile the model with an appropriate loss and accuracy metric.

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    # YOUR CODE HERE: Add loss and metric
)
q_2.check()

In [ ]:
#%%RM_IF(PROD)%%
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)
q_2.assert_check_passed()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_2.hint()
#_COMMENT_IF(PROD)_
q_2.solution()

### 3) Train the Model

In [ ]:
history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=15,
)

Run the cell below to plot the loss and metric curves for this training run.

In [ ]:
import pandas as pd

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();


<!--TODO: Thought question-->

In [ ]:
# View the solution (Run this code cell to receive credit!)
q_3.solution()

# Stacking Convolutional Layers #

### 4)

# Conclusion #
